The following cells represent the essential code of our best model

In [16]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
#print(tf.__version__)

In [17]:
model_name = 'DirectV8'

In [18]:
# Random seed for reproducibility
seed = 44

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [19]:
#importing the dataset
dataset = pd.read_csv('../input/timeseries-training/Training.csv')

In [20]:
# Normalize both features and labels

X_train_raw = dataset.iloc[:]

X_min = X_train_raw.min()
X_max = X_train_raw.max()

X_train_raw = (X_train_raw-X_min)/(X_max-X_min)

In [21]:
#Setting window and stride
window = 200
stride = 20

In [22]:
def build_sequences(df, target_labels, window, stride, telescope):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [23]:
#Define target labels and telescope size
target_labels = dataset.columns
telescope = 864

In [24]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
#X_train.shape, y_train.shape

In [25]:
#define input and output shapes, as well as batch size and number of epochs

input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 300

In [26]:
#defining the model

def build_CONV_LSTM_model(input_shape, output_shape):
    model = tfk.Sequential()
    
    model.add(tfkl.LSTM(input_shape = input_shape, units=10, return_sequences=True))
    model.add(tfkl.Conv1D(20, 3, padding='same', activation='relu'))
    model.add(tfkl.MaxPool1D(pool_size=2))    
    
    model.add(tfkl.Dropout(0.5))
    model.add(tfkl.LSTM(input_shape = input_shape, units=256, return_sequences=True))
    model.add(tfkl.Conv1D(512, 3, padding='same', activation='relu'))
    model.add(tfkl.GlobalAveragePooling1D())    
              
    model.add(tfkl.Dropout(0.5))
    
    model.add(tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu'))
    model.add(tfkl.Reshape((output_shape[-2],output_shape[-1])))
    model.add(tfkl.Conv1D(output_shape[-1], 1, padding='same'))
    model.add(tfkl.Dense(7))

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=[tfk.metrics.RootMeanSquaredError()])
    print("Compiled")
    
    # Return the model
    return model

In [27]:
#building the model

model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [28]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=32, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=16, factor=0.5, min_lr=1e-6)
    ]
).history

In [29]:
#Find and print on the screen the best epoch, loss and rmse

best_epoch = np.argmin(history['val_loss'])
best_epoch_val_loss = history['val_loss'][best_epoch]
best_epoch_val_rmse = history['val_root_mean_squared_error'][best_epoch]

print("Best epoch: " + str(best_epoch+1) + " - val_loss " + str(round(best_epoch_val_loss,5)) + " - val_root_mean_squared_error " + str(round(best_epoch_val_rmse,5)))

In [30]:
model.save(model_name)